In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install transformers
!pip install swifter
!pip install dgl-cu100

In [0]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import time
import gc
from glob import glob
from tqdm import tqdm_notebook as tqdm
import pickle
import logging

import base64
import swifter
import gensim

from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset

pd.set_option('display.max_columns', 500)

In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

import keras
from keras.preprocessing.sequence import pad_sequences

import networkx as nx
import dgl
import dgl.function as fn
from dgl.nn.pytorch import GraphConv

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,DistilBertTokenizer,DistilBertModel
)

from tqdm import tqdm
print(transformers.__version__)

Using TensorFlow backend.


2.8.0


Using backend: pytorch


In [0]:
os.getcwd()

'/content'

In [0]:
DATA_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/data/'
LOG_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/log/'
W2V_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/w2v/'
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/model/'

In [0]:
# DATA_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/data/'
# LOG_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/log/'
# W2V_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/w2v/'
# BERT_DIR = 'D:/project/ICF_AutoCapsule_disabled/BERT/'
# MODEL_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/model/'

### Read Data

In [0]:
train = pd.read_csv(DATA_DIR+'train/train.sample.tsv', sep='\t')
print(train.shape)
train.head()

(10000, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,102851856,338,209,2,AAAAAAAAQEAAAKlDAABPQwAAb0MAALBBAICmQwAATEM=,AAAAAAAAAAAnNsFAAAAAAOeVGD0AAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAcAAAAAAAAAA==,treble popular reed,923271
1,101783080,80,61,1,AACgQAAAoEAAAJZCAABgQg==,AAAAADnKfjyatb8/AAAAALxPBz0MH6E9xwkoPAAAAAAAAA...,HAAAAAAAAAA=,compatible ink cartridges,234031
2,100492797,750,750,1,AAAsQgCAg0MAgCREAADqQw==,AAAAAAAAAABmUlE/AAAAABvc0jwAAAAAAAAAAAAAAAAAAA...,BwAAAAAAAAA=,check new look facial cream,199926
3,101708088,480,640,5,AAB6QwCA0kMAANBDAMAeRAAAfEMAAKBAAIDWQwAAVkMAAH...,AAAAAAAAAAAAAAAAAAAAAAXyfz4AAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAcAAAAAAAAABwAAAAAAAAAHAAAAAAAAAAcAA...,royal commemorative stamp,772725
4,100316842,800,800,4,AAAKQwCAgEMAADNEAAAHRAAAIkMAwCZEAACqQwDARkQAAE...,AAAAADSdaTwVTdA8AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAGAAAAAAAAAAYAAAAAAAAAHAAAAAAAAAA=,calendula baby shower gel,362576


In [0]:
valid = pd.read_csv(DATA_DIR+'valid/valid.tsv', sep='\t')
print(valid.shape)
valid.head()

(14720, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,103047314,800,800,2,AACoQgAA5EIAAKdDAIDuQwCAq0MAAJBCAMA3RABAHkQ=,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0
1,103030678,800,800,9,AIDRQwAAoEAAwEJEAABpQwAAA0MAgMlDAABHRAAAIkQAgL...,AAAAAAAAAACdFBI7AAAAAAAAAABu+j0+AAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAAQAAAAAAAAAHQAAAAAAAAAEAA...,breathable and comfortable children's shoes,0
2,103031217,488,750,2,AABGQwAANEIAgNNDAADsQwAAUEIAALVDAADRQwCALEQ=,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0
3,103026621,929,800,10,AACtQwCAA0QAgPxDAIAoRAAAekMAALdDAACwQwCA30MAgK...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAdAAAAAAAAAAAAAAAAAAAACQAAAAAAAAAJAA...,breathable and comfortable children's shoes,0
4,103017048,800,800,2,AAA+QwCAi0MAgAtEAMA0RAAAqEMAAHhCAMA1RABADUQ=,AAAAAAAAAADIb2s9AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0


In [0]:
testA = pd.read_csv(DATA_DIR+'testA/testA.tsv', sep='\t')
print(testA.shape)
testA.head()

(28830, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,103052445,798,798,1,AIC3QwAAF0MAABxEAMAtRA==,AAAAAAAAAABwKQE9AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0
1,103044513,800,800,1,AABqQwAAzEIAADJEAMAgRA==,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0
2,103035030,800,800,8,AAAAQwAAAEEAgNRDAICGQwAAUEEAAPBBAACUQgAA3EIAAI...,AAAAAAAAAABFkBg+AAAAAM5daTwAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAFAAAAAAAAAAIAAAAAAAAAAwAAAAAAAAAEAA...,straw heavy-bottomed sandals,0
3,103036197,800,800,3,AAAmQwAAwEEAwC1EAICqQwAAJ0MAAFJDAAA1RADAAkQAAE...,AAAAAAAAAAD2rhE+AAAAAAAAAADbLAw+AAAAANGzLD4AAA...,BAAAAAAAAAAEAAAAAAAAAAQAAAAAAAAA,straw heavy-bottomed sandals,0
4,103055323,800,800,1,AABUQwAA+EEAgChEAEBCRA==,AAAAAKzRGTs0osc+kJpVPAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0


### Decode Data

In [0]:
def transform_boxes(df_row):
    return np.frombuffer(base64.b64decode(df_row.boxes), dtype=np.float32).reshape(df_row.num_boxes, 4)

def transform_features(df_row):
    return np.frombuffer(base64.b64decode(df_row.features), dtype=np.float32).reshape(df_row.num_boxes, 2048)

def transform_class_labels(df_row):
    return np.frombuffer(base64.b64decode(df_row.class_labels), dtype=np.int64).reshape(df_row.num_boxes)

def transform_df(df):
    df.boxes = df.swifter.apply(transform_boxes, axis=1)
    df.features = df.swifter.apply(transform_features, axis=1)
    df.class_labels = df.swifter.apply(transform_class_labels, axis=1)
    return df

In [0]:
train = transform_df(train)
valid = transform_df(valid)
testA = transform_df(testA)

### Feature Engineering

- box area

In [0]:
# box area
train['box_area_normalized'] = train.apply(lambda x: np.array([(xi[3]-xi[1])/x.image_w * (xi[2]-xi[0])/x.image_h for xi in x.boxes]), axis=1)
valid['box_area_normalized'] = valid.apply(lambda x: np.array([(xi[3]-xi[1])/x.image_w * (xi[2]-xi[0])/x.image_h for xi in x.boxes]), axis=1)
testA['box_area_normalized'] = testA.apply(lambda x: np.array([(xi[3]-xi[1])/x.image_w * (xi[2]-xi[0])/x.image_h for xi in x.boxes]), axis=1)

- word2vec of box labels

In [0]:
class_label_dict = {
    0:'top clothes (coat, jacket, shirt, etc.)',
    1:'skirt & dress',
    2:'bottom clothes (trousers, pants, etc.)',
    3:'luggage, leather goods',
    4:'shoes',
    5:'accessories (jewelry, clothing accessories, belts, hats, scarves, etc.)',
    6:'snacks, nuts, liquor and tea',
    7:'makeup, perfume, beauty tools and essential oils',
    8:'bottle drink',
    9:'furniture',
    10:'stationery',
    11:'household electrical appliances',
    12:'home decoration',
    13:'household fabric',
    14:'kitchenware',
    15:'home / personal cleaning tools',
    16:'storage supplies',
    17:'motorcycle, motorcycle accessories, vehicles, bicycle and riding equipment',
    18:'outdoor product',
    19:'lighting',
    20:'toys',
    21:'underwear',
    22:'digital supplies',
    23:'bed linens',
    24:'baby products',
    25:'personal care',
    26:'sporting goods',
    27:'clothes (accessories, baby clothing, etc.)',
    28:'others',
    29:'human face',
    30:'arm',
    31:'hair',
    32:'hand',
}

train.class_labels = train.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))
valid.class_labels = valid.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))
testA.class_labels = testA.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))

In [0]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path, encoding='utf-8') as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():  # word_indexのwordに対応するembがあれば、embを代入する
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words


def load_word_embed(word_embed_glove=W2V_DIR+"glove.840B.300d.txt", 
                    word_embed_crawl=W2V_DIR+"crawl-300d-2M.vec",
                    save_filename=W2V_DIR+"word_embedding_matrix_testA",
                    word_index=None):
    
    # Tokneizerの学習
    Tokenizer = keras.preprocessing.text.Tokenizer(filters='', lower=False)
    Tokenizer.fit_on_texts(list(train['query'])+list(valid['query'])+list(testA['query'])
                           +list(train['class_labels'])+list(valid['class_labels'])+list(testA['class_labels']))
    
    if os.path.exists(save_filename + ".npy"):
        embedding_matrix = np.load(save_filename + ".npy").astype("float32")
    else:
        
        if word_index is None:
            word_index = Tokenizer.word_index
        
        glove_matrix, unknown_words_glove = build_matrix(word_index, word_embed_glove)
        print('n unknown words (glove): ', len(unknown_words_glove))
        
        crawl_matrix, unknown_words_crawl = build_matrix(word_index, word_embed_crawl)
        print('n unknown words (crawl): ', len(unknown_words_crawl))
        
        embedding_matrix = crawl_matrix + glove_matrix  
        np.save(save_filename, embedding_matrix)

        del crawl_matrix
        del glove_matrix
        gc.collect()
        
    return embedding_matrix, Tokenizer


def calc_mean_w2v(class_labels):
    class_labels_index = tokenizer_w2v.texts_to_sequences([class_labels])
    try:
        return np.mean(embedding_matrix[class_labels_index], axis=0)
    except KeyError:
        print('Error')
        return np.zeros(300)

In [0]:
# about 5 minites
embedding_matrix, tokenizer_w2v = load_word_embed()

In [0]:
train['class_labels_vec'] = train['class_labels'].swifter.apply(calc_mean_w2v)
valid['class_labels_vec'] = valid['class_labels'].swifter.apply(calc_mean_w2v)
testA['class_labels_vec'] = testA['class_labels'].swifter.apply(calc_mean_w2v)

- Adjaccency matrix

In [0]:
from numba import jit

@jit
def get_adj_mat(X, Y, adj_mat):
    n_box = adj_mat.shape[0]
    if n_box!=1:
        for i in range(n_box):
            for j in range(i+1, n_box):
                d = np.sqrt((X[i] - X[j])**2 + (Y[i] - Y[j])**2)
                adj_mat[i, j] = 1/(d+1e-7)
                adj_mat[j, i] = 1/(d+1e-7)
                
    return adj_mat

def get_adj_mat_row(x):
    n_box = x.num_boxes
    adj_mat = np.zeros((n_box, n_box))
    return get_adj_mat(x.center_normalized[:,0], x.center_normalized[:,1], adj_mat)

In [0]:
# top/left/bottom/right
train['center_normalized'] = train.apply(lambda x: np.array([[(xi[3]-xi[1])/x.image_w, (xi[2]-xi[0])/x.image_h] for xi in x.boxes]), axis=1)
valid['center_normalized'] = valid.apply(lambda x: np.array([[(xi[3]-xi[1])/x.image_w, (xi[2]-xi[0])/x.image_h] for xi in x.boxes]), axis=1)
testA['center_normalized'] = testA.apply(lambda x: np.array([[(xi[3]-xi[1])/x.image_w, (xi[2]-xi[0])/x.image_h] for xi in x.boxes]), axis=1)

In [0]:
train['adj_mat'] = train.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)
valid['adj_mat'] = valid.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)
testA['adj_mat'] = testA.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)

### Negative Sample生成
- ここでは、簡単のため、queryだけランダムにシャッフルしたデータフレームを作成し、concatする
- ラベルとして、元データはlabel=1, ランダムシャッフルはlabel=0とする

In [0]:
train['label'] = 1

train_neg = train.copy(deep=True)
train_neg['label'] = 0
train_neg['query'] = train_neg['query'].sample(frac=1, random_state=42)

train = pd.concat([train, train_neg], axis=0)
train = train.reset_index(drop=True)
print(train.shape)

(20000, 14)


In [0]:
valid['label'] = 1

## Model

- initial setting

In [0]:
# make logger
logger = logging.getLogger('main')
logger.setLevel(logging.DEBUG)
sc = logging.StreamHandler()
logger.addHandler(sc)
fh = logging.FileHandler(LOG_DIR+'20200503_01.log')
logger.addHandler(fh)

In [0]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


### DataLoader

In [0]:
MAX_LEN = 21

class KDDDataset(Dataset):

    def __init__(self, df, train_mode=True, transform=None):
        self.df = df
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.w2v_tokenizer = tokenizer_w2v
        #self.transform = transform
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        ######################
        # query side
        ######################
        query = row['query']
        inputs_query = self.bert_tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )
        
        ids_query = inputs_query["input_ids"]
        token_type_ids_query = inputs_query["token_type_ids"]
        mask_query = inputs_query["attention_mask"]
        
        padding_len = MAX_LEN - len(ids_query)
        ids_query = ids_query + ([0]*padding_len)
        token_type_ids_query = token_type_ids_query + ([0]*padding_len)
        mask_query = mask_query + ([0]*padding_len)
        
        
        ######################
        # image side
        ######################
        adj_mat = row['adj_mat']
        features = row['features']
        box_area = row['box_area_normalized']
        
        g = dgl.DGLGraph(nx.from_numpy_matrix(adj_mat))
        g.ndata['h'] = torch.tensor(features, dtype=torch.float).to(device)
        g.ndata['w'] = torch.tensor(box_area, dtype=torch.float).to(device)
        
        # class_labels
        class_labels_vec = row['class_labels_vec']
        
        
        # positive, negative label
        labels = row['label']
        
        
        return g, {
            'ids_query': torch.tensor(ids_query, dtype=torch.long),
            'mask_query': torch.tensor(mask_query, dtype=torch.long),
            'token_type_ids_query': torch.tensor(token_type_ids_query, dtype=torch.long),
            'class_labels_vec': torch.tensor(class_labels_vec, dtype=torch.float),
            'label': torch.tensor(labels, dtype=torch.float),
        }         

In [0]:
def collate(samples):
    # input samples は、ペアのリスト
    #  [(graph, {features}),...], バッチサイズの長さのリスト
    # グラフは、隣接行列を対角につなげて1つのバッチにする
    # 他のfeatureは、(バッチサイズ, 次元数)のtensor
    
    graphs, features_dict = map(list, zip(*samples))
    
    batched_graph = dgl.batch(graphs)
    
    batched_features = {}
    labels = []
    for i, feat in enumerate(features_dict):
        if i==0:
            for (key, val) in feat.items():
                if key!='label':
                    batched_features[key] = val.view(1,-1)
                else:
                    labels.append(val.item())
        else:
            for (key, val) in feat.items():
                if key!='label':
                    batched_features[key] = torch.cat([batched_features[key],  val.view(1,-1)], dim=0)
                else:
                    labels.append(val.item())
    
    batched_features['label'] = torch.tensor(labels, dtype=torch.float)
    
    return batched_graph, batched_features

In [0]:
# # dataloaderの挙動確認

# dataset_train = KDDDataset(train, train_mode=True)
# train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)

# for i, (graph, features) in enumerate(train_loader):
#     if i<1:
#         print(graph)
#         print(features['ids_query'].shape)
#         print(features['mask_query'].shape)
#         print(features['token_type_ids_query'].shape)
#         print(features['class_labels_vec'].shape)
#         print(features['label'].shape)

#     else:
#         break

### QueryModel

In [0]:
# class QueryModel(nn.Module):
#     def __init__(self):
#         super(QueryModel, self).__init__()
#         self.model_name = 'QueryModel'
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        
#         self.fc_bert = nn.Linear(768, 128)
#         #self.fc = nn.Linear(256, 128)

#     def forward(self, ids, token_type_ids_query, mask_query):
#         layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=token_type_ids_query, attention_mask=mask_query)
        
#         out = F.avg_pool1d(layers.transpose(1,2), kernel_size=layers.size()[1]).squeeze()
#         out = self.fc_bert(out)        
#         out = F.dropout(out, p=0.2, training=self.training)
#         out = F.relu(out)
        
#         return out 

### ImageModel

In [0]:
# class ImageModel(nn.Module):
#     def __init__(self):
#         super(ImageModel, self).__init__()
#         self.gcn1 = GraphConv(2048, 256)
#         self.gcn2 = GraphConv(256, 128)
        
#         self.fc_emb = nn.Linear(300, 64)

#         self.fc = nn.Linear(128+64, 128)

#     def forward(self, g, class_labels_vec):
#         h = g.ndata['h']
#         h = F.relu(self.gcn1(g, h))
#         h = F.relu(self.gcn2(g, h))
#         g.ndata['h'] = h
        
#         # Calculate graph representation by averaging all the node representations.
#         x = dgl.mean_nodes(g, 'h', 'w')  # wでの重み付き平均
        
#         # box_labels
#         x_emb = self.fc_emb(class_labels_vec)


#         x = torch.cat([x, x_emb], dim=-1)
#         x = F.dropout(x, p=0.2, training=self.training)
#         x = F.relu(self.fc(x))
        
#         return x

## Total Model

In [0]:
class KDDModel(nn.Module):
    def __init__(self):
        super(KDDModel, self).__init__()
        self.model_name = 'KDDModel'

        # query
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.fc_bert = nn.Linear(768, 128)

        # image
        self.gcn1 = GraphConv(2048, 256)
        self.gcn2 = GraphConv(256, 128)
        self.fc_emb = nn.Linear(300, 64)
        self.fc = nn.Linear(128+64, 128)

    def forward(self, ids, token_type_ids_query, mask_query, g, class_labels_vec):
        
        ##############################
        # query
        ##############################
        layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=token_type_ids_query, attention_mask=mask_query)
        out_query = F.avg_pool1d(layers.transpose(1,2), kernel_size=layers.size()[1]).squeeze()
        out_query = self.fc_bert(out_query)        
        out_query = F.dropout(out_query, p=0.2, training=self.training)
        out_query = F.relu(out_query)


        ##############################
        # image
        ##############################
        h = g.ndata['h']
        h = F.relu(self.gcn1(g, h))
        h = F.relu(self.gcn2(g, h))
        g.ndata['h'] = h
        
        # Calculate graph representation by averaging all the node representations.
        out_image = dgl.mean_nodes(g, 'h', 'w')  # wでの重み付き平均
        
        # box_labels
        out_emb = self.fc_emb(class_labels_vec)

        out_image = torch.cat([out_image, out_emb], dim=-1)
        out_image = F.dropout(out_image, p=0.2, training=self.training)
        out_image = F.relu(self.fc(out_image))

        ##############################
        # concat
        ##############################
        out = torch.bmm(out_query.unsqueeze(1), out_image.unsqueeze(2)).squeeze()
        
        return out 

### Training

In [0]:
def train_model(train_loader, model, optimizer, criterion): #, scheduler):
    model.train()
    avg_loss = 0.
    for idx, (graph, features) in enumerate(tqdm(train_loader)):
        ids_query = features['ids_query'].to(device)
        mask_query = features['mask_query'].to(device)
        token_type_ids_query = features['token_type_ids_query'].to(device)
        class_labels_vec = features['class_labels_vec'].to(device)
        labels = features['label'].to(device)

        out = model(ids_query, token_type_ids_query, mask_query, graph, class_labels_vec)
        
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del ids_query,mask_query,token_type_ids_query,class_labels_vec,labels,out,loss

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss


def valid_model(valid_loader, model, criterion):
    model.eval()
    avg_loss = 0.
       
    with torch.no_grad():
        for idx, (graph, features) in enumerate(tqdm(valid_loader)):
            ids_query = features['ids_query'].to(device)
            mask_query = features['mask_query'].to(device)
            token_type_ids_query = features['token_type_ids_query'].to(device)
            class_labels_vec = features['class_labels_vec'].to(device)
            labels = features['label'].to(device)

            out = model(ids_query, token_type_ids_query, mask_query, graph, class_labels_vec)

            loss = criterion(out, labels)
            
            avg_loss += loss.item() / len(valid_loader)
            del ids_query,mask_query,token_type_ids_query,class_labels_vec,labels,out,loss
        
    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

In [43]:
model = KDDModel().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# optimizer = torch.optim.Adam([
#                     {'params': model_image.parameters()},
#                     {'params': model_query.parameters()}
#                 ])

criterion = nn.BCEWithLogitsLoss()


loss_list_epoch_train = []
loss_list_epoch_valid = []
loss_best = 100
for epoch in range(5):

    torch.cuda.empty_cache()
    start_time   = time.time()

    dataset_train = KDDDataset(train, train_mode=True)
    dataset_valid = KDDDataset(valid, train_mode=True)
    train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)
    valid_loader = DataLoader(dataset_valid, batch_size=128, shuffle=False, num_workers=0, drop_last=True, collate_fn=collate)
    #scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps=config.epochs*len(train_loader))

    loss_train = train_model(train_loader, model, optimizer, criterion)#, scheduler)
    loss_valid = valid_model(valid_loader, model, criterion)

    loss_list_epoch_train.append(loss_train)
    loss_list_epoch_valid.append(loss_valid)

    #print(f'Epoch {(epoch+1)}, train_loss: {loss_train}, valid_loss: {loss_valid}, time: {(time.time()-start_time)}')
    logger.info(f'Epoch {(epoch+1)}, train_loss: {loss_train}, valid_loss: {loss_valid}, time: {(time.time()-start_time)}')

    # Eearly Stopping
    if loss_valid < loss_best:
        loss_best = loss_valid
        best_param = model.state_dict()
        torch.save(best_param, MODEL_DIR+f'best_param.pt')
        patience = 0
    else:
        patience += 1
        if patience >= 3:
            del train_loader, valid_loader, loss_train, loss_valid
            torch.cuda.empty_cache()
            gc.collect()
            break

    del train_loader, valid_loader, loss_train, loss_valid
    torch.cuda.empty_cache()
    gc.collect()

100%|██████████| 115/115 [00:37<00:00,  3.09it/s]
Epoch 1, train_loss: 0.6948001743891302, valid_loss: 0.6867685711902117, time: 119.39895391464233
100%|██████████| 115/115 [00:37<00:00,  3.10it/s]
Epoch 2, train_loss: 0.6933082242806747, valid_loss: 0.6913780575213228, time: 120.76893639564514
100%|██████████| 115/115 [00:37<00:00,  3.08it/s]
Epoch 3, train_loss: 0.6932498174600111, valid_loss: 0.6773860578951626, time: 119.27448797225952
100%|██████████| 115/115 [00:36<00:00,  3.12it/s]
Epoch 4, train_loss: 0.6932784070571257, valid_loss: 0.6926286412322004, time: 120.57142186164856
100%|██████████| 115/115 [00:37<00:00,  3.07it/s]
Epoch 5, train_loss: 0.6931443913624841, valid_loss: 0.6923227885495059, time: 119.63573479652405


In [0]:
# def train_model(train_loader, model_image, model_query, optimizer1, optimizer2, criterion): #, scheduler):
#     model_image.train()
#     model_query.train()
#     avg_loss = 0.
#     for idx, (graph, features) in enumerate(tqdm(train_loader)):
#         ids_query = features['ids_query'].to(device)
#         mask_query = features['mask_query'].to(device)
#         token_type_ids_query = features['token_type_ids_query'].to(device)
#         class_labels_vec = features['class_labels_vec'].to(device)
#         labels = features['label'].to(device)

#         out_query = model_query(ids_query, token_type_ids_query, mask_query)
#         out_image = model_image(graph, class_labels_vec)
#         out = torch.bmm(out_query.unsqueeze(1), out_image.unsqueeze(2)).squeeze()
        
#         loss = criterion(out, labels)
#         loss.backward()
#         optimizer1.step()
#         optimizer2.step()
#         optimizer1.zero_grad()
#         optimizer2.zero_grad()
        
#         avg_loss += loss.item() / len(train_loader)
#         del ids_query,mask_query,token_type_ids_query,class_labels_vec,labels,out_query,out_image,out,loss

#     torch.cuda.empty_cache()
#     gc.collect()
#     return avg_loss


# def valid_model(valid_loader, model_image, model_query, criterion):
#     model_image.eval()
#     model_query.eval()
#     avg_loss = 0.
       
#     with torch.no_grad():
#         for idx, (graph, features) in enumerate(tqdm(valid_loader)):
#             ids_query = features['ids_query'].to(device)
#             mask_query = features['mask_query'].to(device)
#             token_type_ids_query = features['token_type_ids_query'].to(device)
#             class_labels_vec = features['class_labels_vec'].to(device)
#             labels = features['label'].to(device)

#             out_query = model_query(ids_query, token_type_ids_query, mask_query)
#             out_image = model_image(graph, class_labels_vec)
#             out = torch.bmm(out_query.unsqueeze(1), out_image.unsqueeze(2)).squeeze()

#             loss = criterion(out, labels)
            
#             avg_loss += loss.item() / len(valid_loader)
#             del ids_query,mask_query,token_type_ids_query,class_labels_vec,labels,out_query,out_image,out,loss
        
#     torch.cuda.empty_cache()
#     gc.collect()
#     return avg_loss

In [0]:
# model_image = ImageModel().to(device)
# model_query = QueryModel().to(device)

# optimizer1 = torch.optim.Adam(model_image.parameters(), lr=1e-4)
# optimizer2 = torch.optim.Adam(model_query.parameters(), lr=3e-5, eps=4e-5)
# # optimizer = torch.optim.Adam([
# #                     {'params': model_image.parameters()},
# #                     {'params': model_query.parameters()}
# #                 ])

# criterion = nn.BCEWithLogitsLoss()


# loss_list_epoch_train = []
# loss_list_epoch_valid = []
# loss_best = 100
# for epoch in range(5):

#     torch.cuda.empty_cache()
#     start_time   = time.time()

#     dataset_train = KDDDataset(train, train_mode=True)
#     dataset_valid = KDDDataset(valid, train_mode=True)
#     train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)
#     valid_loader = DataLoader(dataset_valid, batch_size=128, shuffle=False, num_workers=0, drop_last=True, collate_fn=collate)
#     #scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps=config.epochs*len(train_loader))

#     loss_train = train_model(train_loader, model_image, model_query, optimizer1, optimizer2, criterion)#, scheduler)
#     loss_valid = valid_model(valid_loader, model_image, model_query, criterion)

#     loss_list_epoch_train.append(loss_train)
#     loss_list_epoch_valid.append(loss_valid)

#     #print(f'Epoch {(epoch+1)}, train_loss: {loss_train}, valid_loss: {loss_valid}, time: {(time.time()-start_time)}')
#     logger.info(f'Epoch {(epoch+1)}, train_loss: {loss_train}, valid_loss: {loss_valid}, time: {(time.time()-start_time)}')

#     # Eearly Stopping
#     if loss_valid < loss_best:
#         loss_best = loss_valid
#         best_param_query = model_query.state_dict()
#         best_param_image = model_image.state_dict()
#         torch.save(best_param_query, MODEL_DIR+f'best_param_query.pt')
#         torch.save(best_param_image, MODEL_DIR+f'best_param_image.pt')
#         patience = 0
#     else:
#         patience += 1
#         if patience >= 3:
#             del train_loader, valid_loader, loss_train, loss_valid
#             torch.cuda.empty_cache()
#             gc.collect()
#             break

#     del train_loader, valid_loader, loss_train, loss_valid
#     torch.cuda.empty_cache()
#     gc.collect()